# Model of a movile robot Lego mindstorms EV3 in Gazebo

## Definitions of the model

This model is a model of a [Driving base](https://assets.education.lego.com/v3/assets/blt293eea581807678a/bltdb0d9e7188f73df5/5ec7bfb29b2ffb61d5c8091a/ev3-rem-driving-base.pdf?locale=en-us) with a gyro sensor. This CAD model was created in LeoCAD and modificated with Blender to ajust scale and pose of the model.

In LeoCAD we have a standar unit that is LDU. 1 LDU equivalenten to a 0.04cm with blender we can ajust to the real scale of the model to Gazebo and correct mistakes of the models in Collada and Wavefront format.

In [ ]:
# Calculation of correct scale to have a Waveform model in real scale

proportion_ldu_to_cm = 0.04 # 1 ldu = 0.04 cm
proportion_ldu_to_m = 0.04/100 # 1 ldu = 0.0004 m
print(f"The propotion tho convert LDU to meters is {proportion_ldu_to_m}")

The propotion tho conver LDU to meters is 0.0004


To do collition model in gazebo, I did a equivalence of the model that is a cubiode with 2 cilinders and 1 sphere. The cubideo represents robot chassis and each cilinder represents each one wheel except caster wheel which is represent like a sphere. To do that I use measure tool in blender of the complet model.
<div  align="center">
    <img src="https://imgur.com/otIfWcB.png" alt="Blender mesure tool" width="800px">
</div>
Those measures was verificaded with a vernier caliper in the real model.

In [2]:
# model measures
model_width = 0.154 # m
model_depth = 0.159 # m
model_height = 0.142 # m

# Wheel measures
wheel_diameter = 0.056 # m
wheel_width = 0.028 # m

# Wheel caster
wheel_caster_separtion_axis_wheels = 0.106 # m
wheel_caster_height = 0.032 # m
wheel_caster_diameter = 0.018 # m
wheel_caster_casing_base = 0.004 # m
wheel_caster_casing_cover_sphare = 0.003 # m

# Robot data
wheel_separtion = 0.092 # m
robot_track = wheel_separtion + wheel_width # m
motor_head_radius = 0.024 # m
robot_chasis_separtion_wheels = 0.002 # m
robot_chassis_differential_height = (wheel_diameter - motor_head_radius) / 2 # m
robot_chassis_width = model_width - (2 * (wheel_width + robot_chasis_separtion_wheels)) # m
robot_chassis_depth = model_depth # m
robot_chassis_height = model_height - robot_chassis_differential_height # m
print(f"The robot chasis mesures are: {robot_chassis_width} x {robot_chassis_depth} x {robot_chassis_height} [m]")
print(f"The robot track is {robot_track} [m]")



The robot chasis mesures are: 0.094 x 0.159 x 0.126 [m]
The robot track is 0.12 [m]


## Calulate frames positions respect the robot frame

In [25]:
#Collision frame
collision_frame_x = round((model_depth-wheel_diameter)/2, 3) # m  
collision_frame_y = 0 # m
collision_frame_z = round((model_height-wheel_diameter)/2, 3) # m
collision_position_frame = [collision_frame_x, collision_frame_y, collision_frame_z]
print(f"The collision frame is at {collision_position_frame} [m]")

#Wheel frame
wheel_frame_x = 0 # m
wheel_frame_y = round(robot_track / 2, 3) # m
wheel_frame_z = 0 # m
wheel_left_position_frame = [wheel_frame_x, wheel_frame_y, wheel_frame_z]
wheel_right_position_frame = [wheel_frame_x, -wheel_frame_y, wheel_frame_z]
print(f"The wheel left frame is at {wheel_left_position_frame} [m]")
print(f"The wheel right frame is at {wheel_right_position_frame} [m]")

#Wheel caster frame
wheel_caster_frame_x = - wheel_caster_separtion_axis_wheels # m
wheel_caster_frame_y = 0 # m
wheel_caster_frame_z = round(-(wheel_caster_height-(wheel_caster_diameter/2) -wheel_caster_casing_base-wheel_caster_casing_cover_sphare), 3) # m
wheel_caster_position_frame = [wheel_caster_frame_x, wheel_caster_frame_y, wheel_caster_frame_z]
print(f"The wheel caster frame is at {wheel_caster_position_frame} [m]")

The collision frame is at [0.052, 0, 0.043] [m]
The wheel left frame is at [0, 0.06, 0] [m]
The wheel right frame is at [0, -0.06, 0] [m]
The wheel caster frame is at [-0.106, 0, -0.016] [m]


## Calculete inertia mass moments

To this secction it is importan mesure the mass of each element of the robot, to that I mesure with a gramer the robot part with the follow masses.


In [57]:
robot_chassis_mass = 0.536 # kg
wheel_mass = 0.024 # kg
wheel_caster_mass = 0.024 # kg

### Import libreries

In [81]:
import numpy as np

### Calculete robots chasis cuboid inertia

In [85]:
def moment_of_inertia_cuboid(mass, width, height, depth):
    """
    Calculates the moment of inertia matrix of a cuboid with respect to its center of mass.

    Parameters:
        mass (float): Mass of the cuboid.
        width (float): Width (w) of the cuboid.
        height (float): Height (h) of the cuboid.
        depth (float): Depth (d) of the cuboid.

    Returns:
        np.ndarray: 3x3 moment of inertia matrix.
    """
    I_x = (1/12) * mass * (height**2 + depth**2)
    I_x = round(I_x, 9)
    I_y = (1/12) * mass * (width**2 + height**2)
    I_y = round(I_y, 9)
    I_z = (1/12) * mass * (width**2 + depth**2)
    I_z = round(I_z, 9)


    # Inertia matrix at the center of mass (body-aligned coordinate system)
    inertia_matrix = np.array([
        [I_x, 0, 0],
        [0, I_y, 0],
        [0, 0, I_z]
    ])
    
    return inertia_matrix

I_matrix = moment_of_inertia_cuboid(robot_chassis_mass, robot_chassis_width, robot_chassis_height, robot_chassis_depth)
print("Moment of inertia matrix of the cuboid [kg ]:")
print(I_matrix)


Moment of inertia matrix of the cuboid [kg ]:
[[1.838e-03 0.000e+00 0.000e+00]
 [0.000e+00 1.104e-03 0.000e+00]
 [0.000e+00 0.000e+00 1.524e-03]]


### Calculate inertia cylinders

In [80]:
def moment_of_inertia_cylinder(mass, radius, height):
    """
    Calculates the moment of inertia matrix of a cylinder with respect to its center of mass.

    Parameters:
        mass (float): Mass of the cylinder.
        radius (float): Radius of the cylinder.
        height (float): Height of the cylinder.

    Returns:
        np.ndarray: 3x3 moment of inertia matrix.
    """
    I_x = I_y = (1/12) * mass * (3 * radius**2 + height**2)
    I_z = (1/2) * mass * radius**2

    # Round values (optional)
    I_x = I_y = round(I_x, 9)
    I_z = round(I_z, 9)

    # Inertia matrix at the center of mass
    inertia_matrix = np.array([
        [I_x, 0, 0],
        [0, I_y, 0],
        [0, 0, I_z]
    ])
    
    return inertia_matrix

I_matrix = moment_of_inertia_cylinder(wheel_mass, (wheel_diameter/2), wheel_width)
print("Moment of inertia matrix of the cylinder [kg*m^2]:")
print(I_matrix)


Moment of inertia matrix of the cylinder [kg*m^2]:
[[6.272e-06 0.000e+00 0.000e+00]
 [0.000e+00 6.272e-06 0.000e+00]
 [0.000e+00 0.000e+00 9.408e-06]]


### Calculate inertia sphere

In [76]:
def moment_of_inertia_sphere(mass, radius):
    """
    Calculates the moment of inertia matrix of a solid sphere with respect to its center of mass.

    Parameters:
        mass (float): Mass of the sphere.
        radius (float): Radius of the sphere.

    Returns:
        np.ndarray: 3x3 moment of inertia matrix.
    """
    I = (2/5) * mass * radius**2
    I = round(I, 9)

    # Inertia matrix at the center of mass
    inertia_matrix = np.array([
        [I, 0, 0],
        [0, I, 0],
        [0, 0, I]
    ])
    
    return inertia_matrix

I_matrix = moment_of_inertia_sphere(wheel_caster_mass, (wheel_caster_diameter/2))
print("Moment of inertia matrix of the sphere [kg*m^2]:")
print(I_matrix)

Moment of inertia matrix of the sphere [kg*m^2]:
[[7.780e-07 0.000e+00 0.000e+00]
 [0.000e+00 7.780e-07 0.000e+00]
 [0.000e+00 0.000e+00 7.780e-07]]


## Limits velocity robot 

it is important establish the limits that have our motors

<div align="center">

| Feature                   | EV3 Large Motor (45502) | EV3 Medium Motor (45503) |
|---------------------------|------------------------|------------------------|
| **No-load speed**         | ~1050 RPM             | ~1560 RPM             |
| **Maximum torque**        | ~20 N·cm              | ~8 N·cm               |
| **Current draw (no load)**| ~50 mA                | ~50 mA                |
| **Current draw (max load)** | ~1.3 A               | ~0.8 A                |
| **Encoder resolution**    | 1° per pulse (360/rev) | 1° per pulse (360/rev) |
| **Weight**                | ~80 g                 | ~50 g                 |
| **Common applications**   | Heavy-duty robots, robotic arms | Fast rotations, lightweight mechanisms, robotic grippers |
| **Advantages**            | High torque, good precision | Faster, lightweight, and compact |
| **Disadvantages**         | Lower speed, higher power consumption | Lower torque, less precision under high load |
| **Best for**              | Robots needing strength and precision | Robots needing speed and agility |

</div>

this robot uses two large motors, we will work with a maximun speed of 600 rpm in each motor

In [6]:
import math

def rpm_to_rad_per_sec(rpm):
    return rpm * (math.pi / 30)

v_right_rpm = 600 # rpm
v_left_rpm = 600 # rpm

v_right = rpm_to_rad_per_sec(v_right_rpm) # rad/s
v_left = rpm_to_rad_per_sec(v_left_rpm) # rad/s

linear_x = ((wheel_diameter/2) / 2) * (v_right + v_left)
angular_z = ((wheel_diameter/2) /robot_track) * (v_right - (-1*v_left))
print(f"maximun linear velocity: {linear_x} m/s \nmaximun angular velocity: {angular_z} rad/s")

maximun linear velocity: 1.7592918860102842 m/s 
maximun angular velocity: 29.321531433504735 rad/s
